<a href="https://colab.research.google.com/github/jukdang/AI_repo/blob/main/Dacon_toursism%20data%20classification/travel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

from kobert_tokenizer import KoBERTTokenizer
from kobert import get_pytorch_kobert_model
import torch
from torch import nn
#from transformers import BertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-372js4u1/kobert-tokenizer_f8086e6f5eb2427b8e55a0d8093bc918
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-372js4u1/kobert-tokenizer_f8086e6f5eb2427b8e55a0d8093bc918
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=cf77369e41c3186adaa4e4b555cc2f327c665d26c57c624f786bd5c7e18e1668
  Stored in directory: /tmp/pip-ephem-wheel-cache-3lqocm3v/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l1l92ecx
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-bui

In [3]:
from google.colab import files
import io
uploaded = files.upload()
train_df = pd.read_csv(io.StringIO(uploaded['train_processed2.csv'].decode('UTF-8')))

Saving train_processed2.csv to train_processed2.csv


In [46]:
train_df

,id,resized_img_path,overview,sentence_length,category1,category2,category3,tokenized_overview
0,TRAIN_00000,./image/resized_train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,217,0,0,0,"['소안항', '은', '조용한', '섬', '으로', '인근', '해안이', '청..."
1,TRAIN_00001,./image/resized_train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,244,1,1,1,"['경기도', '이천시', '모가면', '에', '있는', '골프장', '으로', ..."
2,TRAIN_00002,./image/resized_train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,160,2,2,2,"['금오산성', '숯불', '갈비', '는', '한우', '고기', '만을', '전..."
3,TRAIN_00003,./image/resized_train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,67,2,2,2,"['철판', '위', '에서', '요리', '하는', '안동찜닭', '을', '맛볼..."
4,TRAIN_00004,./image/resized_train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,63,2,2,2,"['※', '영업', '시간', '10:30', '~', '20:30', '\n\n..."
...,...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/resized_train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,50,2,2,2,"['해발', '12000', 'm', '에', '자리', '한', '식담', '겸'..."
16982,TRAIN_16982,./image/resized_train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,471,4,7,11,"['설악힐', '호텔', '은', '동해', '고속도로', '속초', '톨게이트',..."
16983,TRAIN_16983,./image/resized_train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,282,4,7,11,"['충남', '서산시', '중심', '가에', '위치', '한', '줌', '모텔'..."
16984,TRAIN_16984,./image/resized_train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,158,1,1,3,"['토', '토큰', '바위', '캠핑장', '은', '경기도', '가평', '지역..."


In [48]:
len(train_df['category1'].unique())

6

In [59]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

        bert_model, vocab  = get_pytorch_kobert_model()
        self.bert = bert_model

        self.LinearModel = nn.Sequential(
            nn.Linear(768, 384, bias=True),
            nn.ReLU(),
            nn.Linear(384, 192, bias=True),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(192, 32, bias=True),
            nn.ReLU(),
            nn.Linear(32,6, bias=True),
            nn.Softmax(dim=1)
        )

    def forward(self, sentence):
        inputs = tokenizer.batch_encode_plus([sentence])
        sequence_output, pooled_output = self.bert(input_ids = torch.tensor(inputs['input_ids']), attention_mask = torch.tensor(inputs['attention_mask']))
        outputs = self.LinearModel(pooled_output)
        return outputs

In [60]:
model = NeuralNetwork()
print(model)

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
NeuralNetwork(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Lin

In [61]:
X = train_df['overview'][1]
pred = model(X)
print(pred)

tensor([[0.1473, 0.1854, 0.1549, 0.1513, 0.1790, 0.1820]],
       grad_fn=<SoftmaxBackward0>)


In [1]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

NameError: ignored